In [38]:
import tkinter as tk
from tkinter import messagebox
import customtkinter as ctk
from tkinter import PhotoImage, ttk
from PIL import Image
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import numpy as np
from scipy.stats import norm
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from plotly.offline import plot
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from itertools import chain

In [2]:
Data_input = pd.read_excel(r'C:\Users\joris\Toegepaste Wiskunde\Toegepaste wiskunde\Jaar 4\Module\DataV1.xlsx',)
Data_output = pd.read_excel(r'C:\Users\joris\Toegepaste Wiskunde\Toegepaste wiskunde\Jaar 4\Module\DataV1.xlsx',sheet_name='LungeTime')



y = Data_output['Time post'][Data_input['Reconstruction']=='Hamstring']
WorkingData = Data_input[Data_input['Reconstruction']=='Hamstring']

WorkingData = WorkingData.iloc[: , :-1]

WorkingData = pd.get_dummies(WorkingData, columns = ['Sex'])
WorkingData = WorkingData.drop(['ID','WeightPost'],axis=1)



#WorkingData[['Age','Height','Weight']] = np.log(WorkingData[['Age','Height','Weight']])
WorkingData['Sex_F'] = WorkingData['Sex_F'].astype(int)
WorkingData['Sex_M'] = WorkingData['Sex_M'].astype(int)
WorkingData['Height'] = WorkingData['Height']*100
X = WorkingData


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=2)


In [3]:
model = Sequential()
model.add(Dense(10, input_dim=5, activation='sigmoid'))
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='SGD', loss='mean_squared_error')

X_train = np.asarray(X_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')

history = model.fit(X_train, y_train, epochs=250, batch_size=10, verbose=0)

print(model.summary())
predictions = model.predict(X_test)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                60        
                                                                 
 dense_1 (Dense)             (None, 8)                 88        
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 157
Trainable params: 157
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 139ms/step


In [138]:
def validate_numeric_input(input_text):
        return input_text.isdigit() or (input_text == "" and not entry_var.get())

    
LungeData =pd.read_excel(r'C:\Users\joris\Toegepaste Wiskunde\Toegepaste wiskunde\Jaar 4\Module\data lunge time.xlsx')
LungeJong = LungeData[LungeData['Age'] <= 25]['Lunge time']
LungeOud = LungeData[LungeData['Age'] > 25]['Lunge time']

def run_button_clicked():
    # Variables from input
    gender = gender_var.get()
    weight = weight_var.get()
    length = length_var.get()
    age = age_var.get()

    # Check voor nummers
    if not (weight.replace('.', '', 1).isdigit() and length.replace('.', '', 1).isdigit() and age.isdigit()):
        messagebox.showerror("Error", "Please enter numeric values for Weight, Length, and Age.")
        return

    patient_name = name_var.get()
    name_display_label.config(text=f"{patient_name}")
    genderInput = [0,1]
    # format de data voor de input van NN, waarschijnlijk dit aanpassen in juiste format
    if gender == "F":
        genderInput = [1,0]
    
    input_data = [float(weight), float(length), float(age)]
    input_data = input_data + genderInput
    Input = np.array([input_data],dtype='f')
        
    # prediction maken van de patient
    try:
        prediction = model.predict(Input)
        Output = np.round(prediction[0][0],2)
        print(Output)
        #output label van NN maken
        output_label.config(text=f"Lunge time in seconds is: {Output:.4f}")
        
    except Exception as e:
        print(f"Error during prediction: {e}")
        messagebox.showerror("Error", "An error occurred during prediction")
        
    #grafiek maken
    x_weights = np.linspace((float(weight)-10),(float(weight)+10), 101)
    x_weights_array = []
    y_lunge = []
    Temp = []
    
    for i in x_weights:
        new_array = [i]
        Temp = [float(age),float(length),i]
        Temp += genderInput
        x_weights_array.append(Temp)

    input_grafiek = np.array(x_weights_array,dtype='f')
    prediction = model.predict(input_grafiek)
    
    output_grafiek = np.round(prediction,9)
    y_lunge = list(chain.from_iterable(output_grafiek))
    print(y_lunge)

    #graph
    figure = Figure(figsize=(5, 4), dpi=100)
    LineGraph = figure.add_subplot(1, 1, 1)
    
    LineGraph.plot(x_weights, y_lunge)
    LineGraph.set_xlabel('Weight (kg)')
    LineGraph.set_ylabel('Predicted Lunge Time (s)')
    LineGraph.set_title('Predicted Lunge Time Based on Different Weights')
    LineGraph.set_ylim([min(y_lunge), max(y_lunge)])
    LineGraph.ticklabel_format(useOffset=False)
    canvas = FigureCanvasTkAgg(figure, master=bottom_frame)
    canvas.draw()
    canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
    
    FigureV2 = Figure(figsize=(5, 4), dpi=100)
    Boxplot = FigureV2.add_subplot(1, 1, 1)
    
    if float(age) > 25:
        Boxplot.boxplot(LungeOud)
    else:
        Boxplot.boxplot(LungeJong)
    Boxplot.set_ylabel('Predicted Lunge Time (s)')
    Boxplot.set_title('Standard lunge time for given age group')
    Boxplot.set_xlabel('IDK')
    canvasBoxplot = FigureCanvasTkAgg(FigureV2, master=left_frame)
    canvasBoxplot.draw()
    canvasBoxplot.get_tk_widget().grid(row=0, column=0, sticky="we")
       
# main window
root = tk.Toplevel()
root.title("Python Dashboard")

# size
root.geometry("1500x900")
root.grid_rowconfigure(0, weight=1)
root.grid_columnconfigure(0, weight=1)
# kleuren, hex en namen
background_color = "#386bd1" 
frame_color = "#b6c5e3"  
button_color = "grey"  
label_color = "black"
label_text_color = "black"

# frames voor de gebieden in de GUI
root.configure(bg=background_color)

left_frame = tk.Frame(root, bg=frame_color)
left_frame.place(relx=0.05, rely=0.45, relwidth=0.4, relheight=0.5)

top_left_frame = tk.Frame(root, bg=frame_color)
top_left_frame.place(relx=0.45, rely=0.05, relwidth=0.25, relheight=0.3)

top_right_frame = tk.Frame(root, bg=frame_color)
top_right_frame.place(relx=0.75, rely=0.05, relwidth=0.2, relheight=0.3)

bottom_frame = tk.Frame(root, bg=background_color)
bottom_frame.place(relx=0.45, rely=0.4, relwidth=0.5, relheight=0.5)

# plaatje, ff de image_path aanpassen 


# gender binary variable
gender_var = tk.StringVar()
gender_label = tk.Label(top_left_frame, text="Sex:", bg=frame_color, fg=label_color, font=("Arial", 14))
gender_label.grid(row=0, column=0, padx=10, pady=10)

male_button = tk.Radiobutton(top_left_frame, text="M", variable=gender_var, value="M", bg=frame_color, fg=label_color, font=("Arial", 12))
male_button.grid(row=0, column=1, padx=(10, 2), pady=10, columnspan=1)

female_button = tk.Radiobutton(top_left_frame, text="F", variable=gender_var, value="F", bg=frame_color, fg=label_color, font=("Arial", 12))
female_button.grid(row=0, column=2, padx=(2, 10), pady=10, columnspan=1)

gender_var.set(" ")


# Entry boxes voor variable: weight, name, length, and age
name_var = tk.StringVar()
name_label = tk.Label(top_left_frame, text="Name:", bg=frame_color, fg=label_color, font=("Arial", 12))
name_label.grid(row=1, column=0, padx=10, pady=10)
name_entry = tk.Entry(top_left_frame, textvariable=name_var, font=("Arial", 11))
name_entry.grid(row=1, column=1, padx=10, pady=10)

name_display_label = tk.Label(left_frame, text="", bg=frame_color, fg=label_color, font=("Arial", 30))
name_display_label.grid(row=1, column=0, padx=10, pady=10)

patient_name = name_var.get()
name_display_label.config(text=f"{patient_name}")

validate_cmd = (root.register(validate_numeric_input), '%P')
weight_var = tk.StringVar()
weight_label = tk.Label(top_left_frame, text="Weight (kg):", bg=frame_color, fg=label_color, font=("Arial", 12))
weight_label.grid(row=2, column=0, padx=10, pady=10)
weight_entry = tk.Entry(top_left_frame, textvariable=weight_var, font=("Arial", 11), validate="key", validatecommand=validate_cmd)
weight_entry.grid(row=2, column=1, padx=10, pady=10)

length_var = tk.StringVar()
length_label = tk.Label(top_left_frame, text="Height (cm):", bg=frame_color, fg=label_color, font=("Arial", 12))
length_label.grid(row=3, column=0, padx=10, pady=10)
length_entry = tk.Entry(top_left_frame, textvariable=length_var, font=("Arial", 11), validate="key", validatecommand=validate_cmd)
length_entry.grid(row=3, column=1, padx=10, pady=10)

age_var = tk.StringVar()
age_label = tk.Label(top_left_frame, text="Age (years):", bg=frame_color, fg=label_color, font=("Arial", 12))
age_label.grid(row=4, column=0, padx=10, pady=10)
age_entry = tk.Entry(top_left_frame, textvariable=age_var, font=("Arial", 11), validate="key", validatecommand=validate_cmd)
age_entry.grid(row=4, column=1, padx=10, pady=10)

# Display for lunge time
output_label = tk.Label(top_right_frame, text="Lunge time in seconds is: ", bg=frame_color, fg=label_text_color, font=("Arial", 14))
output_label.pack(anchor='n', padx=10, pady=10)

# Run button
run_button = tk.Button(root, text="Analyze", bg=button_color, fg="white", font=("Arial", 20), command=run_button_clicked)
run_button.place(relx=0.35, rely=0.1, anchor='center')


root.mainloop()

1/1 [==============================] - 0s 23ms/step
1.75
4/4 [==============================] - 0s 2ms/step
[1.7532527, 1.7532563, 1.7532598, 1.7532632, 1.7532666, 1.7532698, 1.753273, 1.7532761, 1.7532793, 1.7532823, 1.7532853, 1.7532883, 1.7532911, 1.753294, 1.7532969, 1.7532995, 1.7533022, 1.7533048, 1.7533075, 1.75331, 1.7533123, 1.753315, 1.7533174, 1.7533196, 1.7533221, 1.7533244, 1.7533265, 1.7533288, 1.7533311, 1.7533331, 1.7533352, 1.7533373, 1.7533393, 1.7533412, 1.7533433, 1.7533451, 1.753347, 1.7533488, 1.7533507, 1.7533524, 1.7533541, 1.753356, 1.7533575, 1.7533593, 1.7533609, 1.7533624, 1.7533641, 1.7533655, 1.7533672, 1.7533686, 1.7533699, 1.7533715, 1.7533729, 1.7533743, 1.7533755, 1.7533771, 1.7533782, 1.7533795, 1.7533808, 1.753382, 1.7533832, 1.7533844, 1.7533855, 1.7533867, 1.7533879, 1.7533889, 1.7533901, 1.753391, 1.7533922, 1.7533932, 1.7533941, 1.7533953, 1.7533963, 1.7533972, 1.7533981, 1.7533991, 1.7533998, 1.7534008, 1.7534016, 1.7534026, 1.7534034, 1.7534044

0     1.566000
1     2.297667
4     1.477000
6     2.879000
7     1.340333
8     1.452333
10    1.860000
12    1.429667
14    1.280333
15    1.434333
16    1.323333
19    1.865000
20    1.110333
25    2.330000
Name: Lunge time, dtype: float64